In [1]:
import numpy as np
import matplotlib.style
matplotlib.style.use('classic')
#import matplotlib
#matplotlib.use("pdf")
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.integrate import ode
from scipy.interpolate import griddata
from numba import cuda, autojit, jit, vectorize
from pylab import imshow, show
from timeit import default_timer as timer

In [2]:
#@jit(nopython=True)
#@vectorize
def kuramoto_2nd_order( t, x, P, K, alfs ):
    N = int(len( P ))
    theta = x[:N]
    dot_theta = x[N:]
    dotdot_theta = np.zeros( 2*N, np.float64 )
    dotdot_theta[0:N] = dot_theta
    dotdot_theta[N:] = P - np.multiply( alfs, dot_theta ) + np.sum(K * np.sin( np.repeat( theta.reshape(N,1).T, N, axis=0 ) - np.repeat( theta.reshape(N,1), N, axis=1 ) ), axis=1)

    return dotdot_theta

In [91]:
def kuramoto_run(x0, K, P, Alf, t_fin):
    
    N = len(P)
    solver = ode( kuramoto_2nd_order )
    solver.set_integrator('vode', method = 'bdf', order = 5, nsteps=3000)
    solver.set_f_params(P, K, Alf)
    solver.set_initial_value(x0, 0)

    i = 0
    states = []
    t = []
    while solver.successful() and solver.t < t_fin:
        solver.integrate(1, step=True) 
        states.append( solver.y )
        t.append(solver.t)
        i += 1

    t = np.array(t)
    states = np.array(states)
    
    phases = ( states[-1, 0:N] + np.pi) % (2 * np.pi ) - np.pi
    phase_vels = states[-1, N:]
    
    end_state = np.concatenate( [phases, phase_vels] )
    
#     plt.figure()
#     plt.subplot(121)
#     plt.plot(t, states[:,:N])
#     plt.subplot(122)
#     plt.plot(t, states[:,N:])
#     plt.show()
    
    return end_state


In [92]:
def synch_condition( K, w ):
    '''
    Returns an approximation to the critical coupling and to the steady state
    of the system, calculated from topological considerations.
    '''
    G = nx.from_numpy_matrix(K)
    L_dagger = np.linalg.pinv( nx.laplacian_matrix( G ).todense() )
    B = nx.incidence_matrix( G, oriented = True ).todense()
    theta_ss = np.matmul(L_dagger, w) 
    x0 = np.concatenate((theta_ss, 0*theta_ss), axis=1).T
    x0 = x0.flatten()
    x0 = x0.tolist()
    x0 = x0[0]
    k_crit = np.linalg.norm( np.matmul( B.T, theta_ss.T ), np.inf )

    return k_crit, x0

In [88]:
def single_node_bs( kth, K, P, Alf, t_fin, num_angs, num_vels ):
    N = len(P)
    k_crit, x0 = synch_condition( K, P )
    x0 = kuramoto_run( x0, K, P, Alf, t_fin )
    y0 = np.copy( x0 )
    state_returns = 0.0
    state_totals = num_angs*num_vels
    
    for ang_k in np.linspace(0, 2*np.pi, num_angs):
        for velang_k in np.linspace(-0.5, 0.5, num_vels):
            y0[kth] = ang_k
            y0[kth+N] = velang_k
            yfin = kuramoto_run( y0, K, P, Alf, t_fin )
            error_traj = np.linalg.norm( yfin - x0 )
            
            #print(yfin, '\n', error_traj)
            
            if (error_traj < 20):
                state_returns = state_returns + 1.0
                print(ang_k, velang_k)
                
    BS = state_returns/state_totals
    
    return BS, x0

In [89]:
# K = 8*np.array([ [0.0, 1.0, 1.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0] ])
# P = np.array([ 1.0, -1.0, 0.0 ] )
# Alf = np.zeros(3)

K = np.loadtxt( 'params_COL/K_Colombia_pu.txt' )
P = np.loadtxt( 'params_COL/P_Colombia_pu.txt' )
Alf = 0.1*np.ones( P.shape )


t_fin = 200
kth_node = 0
num_angs = 5
num_vels = 5


In [93]:
BS, x0 = single_node_bs( kth_node, K, P, Alf, t_fin, num_angs, num_vels )


0.0 -0.5
0.0 -0.25
0.0 0.0
0.0 0.25
0.0 0.5
1.5707963267948966 -0.5
1.5707963267948966 -0.25
1.5707963267948966 0.0
1.5707963267948966 0.25
1.5707963267948966 0.5
3.141592653589793 -0.5
3.141592653589793 -0.25
3.141592653589793 0.0
3.141592653589793 0.25
3.141592653589793 0.5
4.71238898038469 -0.5
4.71238898038469 -0.25
4.71238898038469 0.0
4.71238898038469 0.25
4.71238898038469 0.5
6.283185307179586 -0.5
6.283185307179586 -0.25
6.283185307179586 0.0
6.283185307179586 0.25
6.283185307179586 0.5


In [94]:
BS

1.0

In [77]:
x0

array([ 4.17038143e-02, -4.17038143e-02,  5.86197757e-14, -1.76000046e-04,
        1.76000046e-04,  4.62569889e-17])